# CS361 Assignment 1

### Ronald Voivod rvoi918 351763490

## Task 1 (Coding)

* A. Implement the basic decision tree learning procedure as discussed in the lectures. First, you will implement a train procedure for your decision tree algorithm. This train procedure will use the information gain criterion to decide on splits, recursively until you have leaves of only one class. In your report use one or two sentences to discuss the output (the output of the training procedure is the trained decision tree using the training data, which is a representation of the if-then-else rules applied in your splits). You may print out your decision tree (you don't have to, however it might help you discuss the trained trees) (This may be large - consider the best way to print it). 

In [89]:
import pandas as pd
import numpy as np

In [90]:
data = pd.read_csv('adult_train_data.csv')
X_train = data.iloc[:, :-1]
y_train = data.iloc[:, -1]

In [91]:
# This DecisionTree Class will be used for Tasks 1a/1b/1c
class DecisionTree:
    def __init__(self):
        self.tree = None
        self.default = None

    # Task 1a: Compute entropy of a dataset
    def entropy(self, y):
        unique, counts = np.unique(y, return_counts=True)
        probs = counts / counts.sum()
        return -np.sum(probs * np.log2(probs))

    # Task 1a: Compute information gain for a given feature using entropy algorithm
    def information_gain(self, X, y, feature):
        total_entropy = self.entropy(y)
        values, counts = np.unique(X[feature], return_counts=True)
        weighted_entropy = sum(
            (counts[i] / sum(counts)) * self.entropy(y[X[feature] == values[i]])
            for i in range(len(values))
        )
        return total_entropy - weighted_entropy

    # Task 1a: Find the best feature to split on based on information gain
    def best_split(self, X, y):
        best_feature = None
        best_gain = -1
        for feature in X.columns:
            gain = self.information_gain(X, y, feature)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
        return best_feature

    # Task 1a: Build a decision tree using best feature splits
    # Task 1b: Added stopping depth so that tree can be limited in number of features.
    def build_tree(self, X, y, depth=0, stopping_depth=None):
        if len(set(y)) == 1:
            return y.iloc[0]
        if X.empty:
            return y.mode()[0]
        if stopping_depth is not None and depth >= stopping_depth:
            return y.mode()[0]
        best_feature = self.best_split(X, y)
        if best_feature is None:
            return y.mode()[0]
        tree = {best_feature: {}}
        for value in X[best_feature].unique():
            sub_X = X[X[best_feature] == value].drop(columns=[best_feature])
            sub_y = y[X[best_feature] == value]
            tree[best_feature][value] = self.build_tree(sub_X, sub_y, depth + 1, stopping_depth)
        return tree

    # Task 1a: Trains the model
    def train(self, X, y, stopping_depth=None):
        self.tree = self.build_tree(X, y, stopping_depth=stopping_depth)
        self.default = y.mode()[0]

    # Task 1c: Predict DecisionTree's accuracy
    def predict(self, X):
        def traverse_tree(x, tree):
            if not isinstance(tree, dict):
                return tree
            feature = list(tree.keys())[0]
            x_val = x[feature]
            if x_val in tree[feature]:
                return traverse_tree(x, tree[feature][x_val])
            else:
                return self.default
        return X.apply(lambda x: traverse_tree(x, self.tree), axis=1)

    # Task 1a: Prints tree structure
    def print_tree(self, tree=None, indent=""):
        if tree is None:
            tree = self.tree
        if not isinstance(tree, dict):
            print(indent + str(tree))
            return
        for key, value in tree.items():
            print(indent + str(key))
            for sub_key, sub_value in value.items():
                print(indent + "  ├─ " + str(sub_key) + ": ", end="")
                self.print_tree(sub_value, indent + "  │  ")

In [92]:
# This prints the above tree
dt = DecisionTree()
dt.train(X_train, y_train) # No stopping depth, defaults to None
dt.print_tree()

relationship
  ├─ Not-in-family:   │  education
  │    ├─ Bachelors:   │    │  age
  │    │    ├─ 30-39:   │    │    │  hours_per_week
  │    │    │    ├─ 35-40:   │    │    │    │  sex
  │    │    │    │    ├─ Male:   │    │    │    │    │  occupation
  │    │    │    │    │    ├─ Adm-clerical:   │    │    │    │    │    │  fnlwgt
  │    │    │    │    │    │    ├─ <100K:   │    │    │    │    │    │    │  0
  │    │    │    │    │    │    ├─ 100K-200K:   │    │    │    │    │    │    │  0
  │    │    │    │    │    │    ├─ 200K-300K:   │    │    │    │    │    │    │  workclass
  │    │    │    │    │    │    │    ├─ Private:   │    │    │    │    │    │    │    │  0
  │    │    │    │    │    │    │    ├─ Federal-gov:   │    │    │    │    │    │    │    │  1
  │    │    │    │    │    │    ├─ >300K:   │    │    │    │    │    │    │  1
  │    │    │    │    │    ├─ Tech-support:   │    │    │    │    │    │  0
  │    │    │    │    │    ├─ Exec-managerial:   │    │    │    │    │  

Discussion of 1a:

* The decision tree that was built seems to be very overfitted as it accounts for every single feature and decides to split based off of it. This is due to the "This train procedure will use the information gain criterion to decide on splits, recursively until you have leaves of only one class" requirement wherein the tree needed to split based off all the possible features until there were no other features to split off of. This led to a very large and descriptive tree where it seems that it accounts for every possible value in the training dataset and likely cannot generalise well when new data is tested.
<br>
<br>

* B. Implement tree depth control as a means of controlling the model complexity.  In the procedure train implement a parameter stopping_depth. Use the stopping_depth parameter to stop further splits of the tree. In your report use one or two sentences to discuss the output on the training data at stopping level 2, 3, 4.  You can print out your decision tree. 

In [93]:
# Task 1b: Loop over different stopping depths according to Task 1b
for depth in [2, 3, 4]:
    print(f"\nDecision Tree with Stopping Depth {depth}:\n")
    dt = DecisionTree()
    dt.train(X_train, y_train, stopping_depth=depth)  # Using the stopping_depth parameter
    dt.print_tree()


Decision Tree with Stopping Depth 2:

relationship
  ├─ Not-in-family:   │  education
  │    ├─ Bachelors:   │    │  0
  │    ├─ HS-grad:   │    │  0
  │    ├─ 9th:   │    │  0
  │    ├─ Masters:   │    │  0
  │    ├─ Assoc-acdm:   │    │  0
  │    ├─ Assoc-voc:   │    │  0
  │    ├─ Some-college:   │    │  0
  │    ├─ Doctorate:   │    │  1
  │    ├─ 10th:   │    │  0
  │    ├─ Preschool:   │    │  0
  │    ├─ 11th:   │    │  0
  │    ├─ Prof-school:   │    │  1
  │    ├─ 7th-8th:   │    │  0
  │    ├─ 5th-6th:   │    │  0
  │    ├─ 12th:   │    │  0
  │    ├─ 1st-4th:   │    │  0
  ├─ Husband:   │  education
  │    ├─ Bachelors:   │    │  1
  │    ├─ 11th:   │    │  0
  │    ├─ HS-grad:   │    │  0
  │    ├─ Some-college:   │    │  0
  │    ├─ 7th-8th:   │    │  0
  │    ├─ Doctorate:   │    │  1
  │    ├─ 9th:   │    │  0
  │    ├─ Assoc-acdm:   │    │  0
  │    ├─ Assoc-voc:   │    │  0
  │    ├─ 5th-6th:   │    │  0
  │    ├─ Masters:   │    │  1
  │    ├─ Prof-school:   │    │  

Discussion of 1b:

* It seems that the size of the Decision Tree grows exponentially as the stopping depth increments by one each time, from 2-4. This is likely due to the fact that for each extra feature, it must include each of the separate discrete observations seen inside of the features, e.g. if FeatureA has 3 possibilities and FeatureB has 4 possibilites, then a decision tree with FeatureA => FeatureB would have 12 different possible combinations. Adding FeatureC with 5 possibilities creates 60 possible combinations. Big ~O(2^n) vs Big O(1).
<br>
<br>

* C. Implement a test procedure for your decision tree algorithm. This procedure takes new data and the trained decision tree model (the decision tree you trained in A or one of the decision trees with depth control you trained in B) and returns a prediction for each example in the new data. Run your test data through this procedure, and evaluate your predictions by using at least one of the classifier evaluation metrics studied in the lectures. Describe your results.

In [94]:
test_data = pd.read_csv('adult_test_data.csv')
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]

In [95]:
dt = DecisionTree()
dt.train(X_train, y_train, stopping_depth=3)
predictions = dt.predict(X_test)

In [96]:
print("Distribution in y_test:", np.unique(y_test, return_counts=True))
print("Distribution in predictions:", np.unique(predictions, return_counts=True))

Distribution in y_test: (array([0, 1]), array([11360,  3700]))
Distribution in predictions: (array([0, 1]), array([12260,  2800]))


In [ ]:
def classification_metrics(y_true, y_pred, positive_label=1):
    tp = ((y_true == positive_label) & (y_pred == positive_label)).sum()
    tn = ((y_true != positive_label) & (y_pred != positive_label)).sum()
    fp = ((y_true != positive_label) & (y_pred == positive_label)).sum()
    fn = ((y_true == positive_label) & (y_pred != positive_label)).sum()
    accuracy = (tp + tn) / len(y_true)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return accuracy, precision, recall

accuracy, precision, recall = classification_metrics(y_test, predictions, positive_label=1)

print("Evaluation Metrics on Test Data:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")

Evaluation Metrics on Test Data:
Accuracy:  0.8227
Precision: 0.6839
Recall:    0.5176
hi


## Task 2 (Reflection - not coding)

* A. Discuss what will happen if you decide to change the splitting criterion. Explain the new splitting criterion and how it might change your decision tree.

Changing the splitting criterion from an information gain algorithm (entropy based), to using the Gini index, the decision tree may select different features at different points for splitting. The Gini index measures impurity, like entropy does, but the Gini index tends to favour splits that create purer child tree nodes. Since different splitting criterion evaluates impurity differently, the structure of the decision tree can change. This results in changes to its depth, complexity, and its predictive capability. One criterion may lead to a more balanced/efficient tree which in turn influences accuracy and generalization, depending on the dataset.

* B. Explain whether your test procedure implemented in Task 1-C can indicate whether your tree is over- or underfitting.  

The test procedure implemented in Task 1c can show signs that your tree is over/underfitting. For example, the accuracy evaluation metric at different stopping depths can show how as you approach a specific stopping depth (supposedly three in this assignment is the ideal stopping depth) accuracy increases, and as you increase stopping depth further, accuracy decreases. This seems to indicate that before stopping depth three, the model is underfitting, as the model is still able to increase in accuracy as tree complexity increases, additionally, after stopping depth three, the decision tree seems to be overfitting, as the model decreases in accuracy as the tree complexity increases, showing that the model may not generalise well to new data. While this is a useful metric, a more specific evaluation like cross-validation might be more useful to diagnose over/underfitting conclusively.